In [ ]:
# ============================================================
# 1) Monter Google Drive
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# ============================================================
# 2) Créer le dossier du projet
# ============================================================
import os

project_path = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw"
os.makedirs(project_path, exist_ok=True)

print("Dossier prêt :", project_path)

# ============================================================
# 3) Télécharger ENEDIS avec barre de progression
# ============================================================
import requests
from tqdm import tqdm
import pandas as pd

url_enedis = "https://data.enedis.fr/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/exports/csv"

local_csv = f"{project_path}/enedis_raw.csv"

print("\n📥 Téléchargement ENEDIS en cours...\n")

# --- Download en streaming ---
response = requests.get(url_enedis, stream=True)
file_size = int(response.headers.get("Content-Length", 0))

chunk_size = 1024
progress = tqdm(total=file_size, unit='B', unit_scale=True, desc="Download")

with open(local_csv, "wb") as f:
    for chunk in response.iter_content(chunk_size=chunk_size):
        if chunk:
            f.write(chunk)
            progress.update(len(chunk))

progress.close()
print("✔ Téléchargement terminé :", local_csv)


# ============================================================
# 4) Charger le CSV téléchargé
# ============================================================
print("\n📄 Chargement du CSV dans pandas...")

enedis = pd.read_csv(local_csv, sep=';')
print("✔ ENEDIS chargé :", enedis.shape)

enedis.head()


Mounted at /content/drive
Dossier prêt : /content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw

📥 Téléchargement ENEDIS en cours...



Download: 424MB [08:07, 870kB/s]


✔ Téléchargement terminé : /content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/enedis_raw.csv

📄 Chargement du CSV dans pandas...


/tmp/ipython-input-3891177186.py:52: DtypeWarning: Columns (1,15) have mixed types. Specify dtype option on import or set low_memory=False.
  enedis = pd.read_csv(local_csv, sep=';')


✔ ENEDIS chargé : (2821929, 19)


,annee,code_iris,nom_iris,numero_de_voie,indice_de_repetition,type_de_voie,libelle_de_voie,code_commune,nom_commune,segment_de_client,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,adresse,code_epci,code_departement,code_region,tri_des_adresses
0,2018,930100204,Le Moulin à Vent,52.0,NaN,RUE,ETIENNE DOLET,93010,BONDY,RESIDENTIEL,10,17.704,1.770,3.465,52 RUE ETIENNE DOLET,200054781.0,93.0,11.0,440008
1,2018,930100102,La Terre Saint-Blaise,2.0,NaN,PLACE,EUDEMON,93010,BONDY,RESIDENTIEL,21,53.759,2.560,3.465,2 PLACE EUDEMON,200054781.0,93.0,11.0,440013
2,2018,930100106,Les Merisiers,211.0,NaN,AVENUE,GALLIENI,93010,BONDY,RESIDENTIEL,168,431.999,2.571,3.465,211 AVENUE GALLIENI,200054781.0,93.0,11.0,440028
3,2018,930100404,La Ferme,132.0,NaN,AVENUE,GALLIENI,93010,BONDY,RESIDENTIEL,32,33.584,1.049,3.465,132 AVENUE GALLIENI,200054781.0,93.0,11.0,440031
4,2018,930100302,La Remise à Jorelle,75.0,NaN,AVENUE,GENEVIEVE ANTHONIOZ DE GAULLE,93010,BONDY,RESIDENTIEL,25,152.632,6.105,3.465,75 AVENUE GENEVIEVE ANTHONIOZ DE GAULLE,200054781.0,93.0,11.0,440038


In [3]:
import pandas as pd
import requests
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

BASE = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines?select=numero_dpe%2Cadresse_ban%2Cadresse_brut%2Cnumero_voie_ban%2Cnom_rue_ban%2Cnom_commune_ban%2Ccode_postal_ban%2Ccode_insee_ban%2Cetiquette_dpe%2Cetiquette_ges%2Csurface_habitable_logement%2Cconso_5_usages_ep%2Cconso_5_usages_par_m2_ep%2Cconso_5_usages_ef%2Cconso_5_usages_par_m2_ef%2Ctype_batiment%2Cperiode_construction%2Cannee_construction%2Cdate_etablissement_dpe%2Cdate_fin_validite_dpe%2Cdate_reception_dpe%2Cdate_visite_diagnostiqueur%2Cdate_derniere_modification_dpe%2Cqualite_isolation_enveloppe%2Cqualite_isolation_murs%2Cdeperditions_planchers_bas%2Cqualite_isolation_menuiseries%2Ctype_installation_chauffage%2Ctype_installation_ecs&size=10000"
OUTPUT = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/dpe_sample_2M.csv"

MAX_PAGES = 200
all_batches = []

url = BASE

for i in tqdm(range(MAX_PAGES), desc="Downloading pages"):
    r = requests.get(url)
    if r.status_code != 200:
        print("ERROR:", r.status_code)
        break

    data = r.json()
    results = data.get("results", [])

    if not results:
        print("Fin prématurée.")
        break

    df = pd.json_normalize(results)
    all_batches.append(df)

    # pagination automatique via l'API ADEME
    if "next" not in data or not data["next"]:
        break
    url = data["next"]

# ---- CONCAT + SAVE ----
df_final = pd.concat(all_batches, ignore_index=True)
df_final.to_csv(OUTPUT, index=False)

print("✔ Fini !")
print("Shape:", df_final.shape)
print("Saved:", OUTPUT)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


✔ Fini !
Shape: (2000000, 30)
Saved: /content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/dpe_sample_2M.csv
